In [1]:
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
# --- 1. Persiapan Data Sampel ---
# Membuat DataFrame sampel sesuai dengan struktur kolom yang Anda berikan.
# Dalam implementasi nyata, Anda akan memuat data Anda dari file (misal: pd.read_csv('nama_file.csv')).
data = {
    'userName': ['pengguna_1', 'pengguna_2', 'pengguna_3', 'pengguna_4'],
    'content': [
        'Vaksinasi COVID-19 di kecamatan @sehatselalu berjalan lancar!! 😊 Syukurlah... Semoga pandemi cepat berakhir. Cek info di https://info.sehat/vaksin',
        'PPKM diperpanjang lagi?!?!? 😠 Capek banget sama aturan yg gak jelas #ppkm #covid19... kapan kita bisa hidup normal lagi yaa? 1000 pertanyaan di kepalaku.',
        'Terima kasih nakes! Kalian pahlawan garda terdepan. Perjuangan kalian luar biasa... #respect 👍',
        'Apa benar varian Delta lebih mematikan dari varian sebelumnya? Saya jadi khawatir... Mohon pencerahannya, guys.'
    ],
    'score': [5, 2, 5, 3],
    'reviewCreatedVersion': ['1.0', '1.0', '1.1', '1.1'],
    'at': ['2022-07-12', '2022-07-12', '2022-07-13', '2022-07-14'],
    'Sentimen Akhir': ['Positif', 'Negatif', 'Positif', 'Negatif']
}
df = pd.DataFrame(data)

In [4]:
print("--- Data Awal ---")
print(df[['userName', 'content']].to_markdown(index=False))
print("\n" + "="*50 + "\n")

--- Data Awal ---
| userName   | content                                                                                                                                                    |
|:-----------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| pengguna_1 | Vaksinasi COVID-19 di kecamatan @sehatselalu berjalan lancar!! 😊 Syukurlah... Semoga pandemi cepat berakhir. Cek info di https://info.sehat/vaksin        |
| pengguna_2 | PPKM diperpanjang lagi?!?!? 😠 Capek banget sama aturan yg gak jelas #ppkm #covid19... kapan kita bisa hidup normal lagi yaa? 1000 pertanyaan di kepalaku. |
| pengguna_3 | Terima kasih nakes! Kalian pahlawan garda terdepan. Perjuangan kalian luar biasa... #respect 👍                                                            |
| pengguna_4 | Apa benar varian Delta lebih mematikan dari varian sebelumnya? Saya jadi khawatir... Mohon pencerahannya, guys

In [14]:
# --- 2. Fungsi Pra-pemrosesan Dasar ---
# Fungsi ini melakukan pembersihan awal berdasarkan metodologi penelitian[cite: 88].
def preprocess_basic(text):
    # Mengubah teks menjadi huruf kecil [cite: 89]
    text = text.lower()
    # Menghapus mention (@username) [cite: 88]
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)
    # Menghapus tagar (#hashtag) [cite: 88]
    text = re.sub(r'#\w+', '', text)
    # Menghapus URL [cite: 88]
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Menghapus emoji [cite: 88]
    text = re.sub(r'<.*?>', '', text) # Menghapus tag HTML jika ada
    # Menggunakan regex untuk menghapus karakter non-alfanumerik (termasuk emoji dan simbol) [cite: 88]
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Menghapus angka [cite: 88]
    text = re.sub(r'\d+', '', text)
    # Menghapus spasi berlebih [cite: 88]
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

# Menerapkan pembersihan dasar
df['cleaned_content'] = df['content'].apply(preprocess_basic)

print("--- Setelah Pembersihan Dasar (Basic Cleaning) ---")
print(df[['userName', 'cleaned_content']].to_markdown(index=False))
print("\n" + "="*50 + "\n")

--- Setelah Pembersihan Dasar (Basic Cleaning) ---
| userName   | cleaned_content                                                                                                           |
|:-----------|:--------------------------------------------------------------------------------------------------------------------------|
| pengguna_1 | vaksinasi covid di kecamatan berjalan lancar syukurlah semoga pandemi cepat berakhir cek info di                          |
| pengguna_2 | ppkm diperpanjang lagi capek banget sama aturan yg gak jelas kapan kita bisa hidup normal lagi yaa pertanyaan di kepalaku |
| pengguna_3 | terima kasih nakes kalian pahlawan garda terdepan perjuangan kalian luar biasa                                            |
| pengguna_4 | apa benar varian delta lebih mematikan dari varian sebelumnya saya jadi khawatir mohon pencerahannya guys                 |




In [6]:
# --- 3. Penghapusan Stopwords ---
# Menggunakan Sastrawi untuk daftar stopwords Bahasa Indonesia.
# Langkah ini sesuai dengan metodologi untuk fokus pada kata bermakna[cite: 91].
remover_factory = StopWordRemoverFactory()
stopword_remover = remover_factory.create_stop_word_remover()

def remove_stopwords(text):
    return stopword_remover.remove(text)

# Menerapkan penghapusan stopwords
df['no_stopwords_content'] = df['cleaned_content'].apply(remove_stopwords)

print("--- Setelah Penghapusan Stopwords ---")
print(df[['userName', 'no_stopwords_content']].to_markdown(index=False))
print("\n" + "="*50 + "\n")

--- Setelah Penghapusan Stopwords ---
| userName   | no_stopwords_content                                                                                    |
|:-----------|:--------------------------------------------------------------------------------------------------------|
| pengguna_1 | vaksinasi covid kecamatan berjalan lancar syukurlah semoga pandemi cepat berakhir cek info              |
| pengguna_2 | ppkm diperpanjang capek banget sama aturan yg gak jelas kapan bisa hidup normal yaa pertanyaan kepalaku |
| pengguna_3 | terima kasih nakes kalian pahlawan garda terdepan perjuangan kalian luar biasa                          |
| pengguna_4 | apa benar varian delta lebih mematikan varian sebelumnya jadi khawatir mohon pencerahannya guys         |




In [7]:
# --- 4. Stemming (Mengubah ke Kata Dasar) ---
# Proses ini mengubah kata ke bentuk dasarnya (misal: 'berjalan' -> 'jalan').
# Tujuannya adalah untuk mengurangi dimensi fitur untuk analisis ML[cite: 94, 95].
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

def stem_text(text):
    return stemmer.stem(text)

# Menerapkan stemming
df['stemmed_content'] = df['no_stopwords_content'].apply(stem_text)

print("--- Setelah Stemming (Hasil Akhir Pra-pemrosesan) ---")
print(df[['userName', 'stemmed_content']].to_markdown(index=False))
print("\n" + "="*50 + "\n")

--- Setelah Stemming (Hasil Akhir Pra-pemrosesan) ---
| userName   | stemmed_content                                                                           |
|:-----------|:------------------------------------------------------------------------------------------|
| pengguna_1 | vaksinasi covid camat jalan lancar syukur moga pandemi cepat akhir cek info               |
| pengguna_2 | ppkm panjang capek banget sama atur yg gak jelas kapan bisa hidup normal yaa tanya kepala |
| pengguna_3 | terima kasih nakes kalian pahlawan garda depan juang kalian luar biasa                    |
| pengguna_4 | apa benar varian delta lebih mati varian belum jadi khawatir mohon cerah guys             |




In [8]:
print("--- DataFrame Final Setelah Semua Tahapan Pra-pemrosesan ---")
# Menampilkan kolom-kolom relevan dari awal hingga akhir
final_columns = ['userName', 'content', 'stemmed_content', 'Sentimen Akhir']
print(df[final_columns].to_markdown(index=False))

--- DataFrame Final Setelah Semua Tahapan Pra-pemrosesan ---
| userName   | content                                                                                                                                                    | stemmed_content                                                                           | Sentimen Akhir   |
|:-----------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------|:-----------------|
| pengguna_1 | Vaksinasi COVID-19 di kecamatan @sehatselalu berjalan lancar!! 😊 Syukurlah... Semoga pandemi cepat berakhir. Cek info di https://info.sehat/vaksin        | vaksinasi covid camat jalan lancar syukur moga pandemi cepat akhir cek info               | Positif          |
| pengguna_2 | PPKM diperpanjang lagi?!?!? 😠 Capek banget sama aturan yg gak jelas #ppkm #c